# BBM 409 FALL 2022 
# ASSIGNMENT 1 
# YUSUF KOCA : 2200356013
# MUSTAFA EMİR PEKER : 2200356011


PART 1: 

        PROBLEM DEFINITION
Here we have 16 personality class and 10.000 samples. We have 60 attributes for each sample. We need to classify given samples by considering the sample's(or person's) attribute values. We will use KNN classification with and without normalized attribute values.

After classfication we will test our model by looking accuracy ,prescion, recall values.

Here are the libraries that we used during this project

In [11]:
import math
import pandas as pd
import numpy as np

First of all we read the first data set .

In [12]:
#part 1

#read 16p.csv
df = pd.read_csv('subset_16P.csv', encoding='cp1252')
#convert to numpy array
df = df.values

After that we created the global variables that will be used in this project.

First global instance is confusion_matrix. Confusion matrix is used for calculating tp, fn ,tn, fp values. We implemented our version of confusion matrix for 16 personality classes.We used the given mapping to map our values.

Second one is all_values. This variable holds every combination that will be calculated in this project.

predictionlist: This is the data sturcture that will be filled with arrays of predicted heating_load and cooling_load values while the KNN algortihm is working.

weightedpredictionlist: This is the data sturcture that will be filled with arrays of predicted weighted heating and cooling values while the KNN algortihm is working.

For both normal and weighted KNN there is only one function that calculates the normal and weighted predictions. After calculations it puts the values to corresponding data structure

In [13]:
global confusion_matrix
confusion_matrix = [[0 for i in range(16)] for j in range(16)]

global all_values
all_values = [[] for l in range(5)]

global predictionlist , weightedpredictionlist

predictionlist = [None] * (2000)
weightedpredictionlist = [None] * (2000)

global precisionerrorlist, recallerrorlist

precisionerrorlist, recallerrorlist = [], []

For this project we had to shuflle the data to reach unbiased results. For that reason we had to implement a shuffle function that is not a built-in function as said in project description. We used a shuffle techniueqe called fisher-yates shuffle.

Funny note : for some reasons there were some duplicated values in our data set. We were curious about this error and we implemented a duplucation counter method. After some try and error we have fixed the issue :))

In [14]:
#shuffle function from scratch
def shuffle(aux, n):
    #shuffle
    for i in range(n):
        #generate random index
        index = np.random.randint(0, len(aux))
        #swap
        aux[i], aux[index] = aux[index], aux[i]

    count_duplicates(aux)
    return aux

#count duplicates
def count_duplicates(set):
    #count duplicates
    count = 0
    for i in range(len(set)-1):
        if set[i][0] == set[i+1][0]:
            count += 1
    return count

There are the functions that we used to make a complete KNN function for classification.

personalityPredict : This function takes a training_Data , a distance list and a k-value for closest neighbors.After taking these values it calculates the predicted classes for given set.

personalityPredictWeighted: This is function does the same thing with the personalityPredict. The only difference between them is personalityPredictWeighted calculates with weighted distances.

personalityKNN: This function calculates the distance list that will be used in personalityPredict and eePredictWeighted. After calculating the distance list , eeKNN function will call eePredict and personalityPredictWeighted and store their results in
predictionlist and weightedpredictionlist.

In [15]:
#unweighted knn prediction
def personalityPredict(training_Data, distancelist, k):
    personalityCounts = [[0, "ESTJ"], [0, "ENTJ"], [0, "ESFJ"], [0, "ENFJ"], [0, "ISTJ"], [0, "ISFJ"], [0, "INTJ"],
                         [0, "INFJ"],
                         [0, "ESTP"], [0, "ESFP"], [0, "ENTP"], [0, "ENFP"], [0, "ISTP"], [0, "ISFP"], [0, "INTP"],
                         [0, "INFP"]]

    for i in range(0, k):
        for j in personalityCounts:
            if training_Data[distancelist[i][1], 61] == j[1]:
                j[0] += 1

    personalityCounts.sort(reverse=True)

    return personalityCounts[0][1]

#weighted knn prediction
def personalityPredictWeighted(training_Data, distancelist, k):
    personalityCounts = [[0, "ESTJ"], [0, "ENTJ"], [0, "ESFJ"], [0, "ENFJ"], [0, "ISTJ"], [0, "ISFJ"], [0, "INTJ"],
                         [0, "INFJ"],
                         [0, "ESTP"], [0, "ESFP"], [0, "ENTP"], [0, "ENFP"], [0, "ISTP"], [0, "ISFP"], [0, "INTP"],
                         [0, "INFP"]]

    for i in range(0, k):
        for j in personalityCounts:
            if training_Data[distancelist[i][1], 61] == j[1]:
                j[0] += 1/distancelist[i][0]

    personalityCounts.sort(reverse=True)

    return personalityCounts[0][1]

#calculates distance for knn algorithm
def personalityKNN (training_Data, test_Data, k):
    distance = 0.0
    global  predictionlist, weightedpredictionlist

    distancelist = [None] * len(training_Data)
    predictionlist = [None] * len(test_Data)
    weightedpredictionlist = [None] * len(test_Data)
    for i in range(len(test_Data)):
        for j in range(len(training_Data)):
            for l in range(1, 60):
                distance += (test_Data[i, l] - training_Data[j, l])**2
            distance = math.sqrt(distance)
            distancelist[j] = [distance, j]
        distancelist.sort()
        predictionlist[i] = personalityPredict(training_Data, distancelist, k)
        weightedpredictionlist[i] = personalityPredictWeighted(training_Data, distancelist, k)

For this project we test our model with raw and normalized data. For raw part we didn't have to do anything. But for normalization we implemented our normalization method. As said in the project description we implemented a min-max normalization method.

In [16]:
#min max normalization
def min_max_normalization_16p(df):
    min, max = 0, 0
    for i in range(1, 60):
        for j in df:
            if j[i] > max:
                max = j[i]
            elif j[i] < min:
                min = j[i]
        for l in df:
            l[i] = (l[i] - min) / (max - min)
        min, max = 0, 0

    return df

To compare our results with the actual values we had to get the actual values. For that we created some functions.


get_class_type: This function takes a numpy array. After that it retrieves the classtypes of that given set and return them in an array.

convert_classes_to_numbers: This function convert an array of strings to numerical values using the given mapping.We need this function because we need to put predicted and actual values into the confusion matrix .It returns an array of integers.

place_in_confusion_matrix: This function takes arrays of predicted and actual values. It uses actual values to index the rows and it uses predicted values to index the columns.

performance_matrix: This function takes a confusion matrix and calculates the accuracy , precision , recall, precision error, recall error. After calculating all these values it appends precision error and recall error, it appends them to precisionerrorlist and recallerrorlist. The last operation done by this function is to return accuracy, precision ,recall values in an array.


In [17]:
#get class types
def get_class_type(set):
    classTypes = []
    for i in range(len(set)):
        type = set[i][61]
        classTypes.append(type)
    return classTypes


#convert classes to numbers
def convert_classes_to_numbers(array):
    number_array = []
    for i in range(len(array)):
        if array[i] == 'ESTJ':
            number_array.append(0)
        elif array[i] == 'ENTJ':
            number_array.append(1)
        elif array[i] == 'ESFJ':
            number_array.append(2)
        elif array[i] == 'ENFJ':
            number_array.append(3)
        elif array[i] == 'ISTJ':
            number_array.append(4)
        elif array[i] == 'ISFJ':
            number_array.append(5)
        elif array[i] == 'INTJ':
            number_array.append(6)
        elif array[i] == 'INFJ':
            number_array.append(7)
        elif array[i] == 'ESTP':
            number_array.append(8)
        elif array[i] == 'ESFP':
            number_array.append(9)
        elif array[i] == 'ENTP':
            number_array.append(10)
        elif array[i] == 'ENFP':
            number_array.append(11)
        elif array[i] == 'ISTP':
            number_array.append(12)
        elif array[i] == 'ISFP':
            number_array.append(13)
        elif array[i] == 'INTP':
            number_array.append(14)
        elif array[i] == 'INFP':
            number_array.append(15)

    return number_array

#place predicted and actual classes in confusion matrix
def place_in_confusion_matrix(actual, predicted):
    global confusion_matrix
    confusion_matrix = confusion_matrix = [[0 for i in range(16)] for i in range(16)]
    for i in range(len(actual)):
        confusion_matrix[actual[i]][predicted[i]] += 1
    return confusion_matrix

#accuracy etc.
def performance_matrix(int):
    global precisionerrorlist, recallerrorlist

    tp , fn, tn, fp = 0, 0, 0, 0
    accuracy, precision, recall = 0, 0, 0
    precisionerror = 0
    recallerror = 0

    for referance in range(16):
        for actual in range(16):
            for prediction in range(16):
                if ((referance == actual) and (referance == prediction)):
                    tp += int[actual][prediction]
                elif ((referance != actual) and (referance == prediction)):
                    fp += int[actual][prediction]
                elif ((referance == actual) and (referance != prediction)):
                    fn += int[actual][prediction]
                elif ((referance != actual) and (referance != prediction)):
                    tn += int[actual][prediction]

        accuracy += (tp + tn) / (tp + tn + fp + fn)
        if tp + fp > 0:
            precision += tp / (tp + fp)
        else:
            precisionerror += 1
        if tp+fn > 0:
            recall += tp / (tp + fn)
        else:
            recallerror += 1
        tp, fn, tn, fp = 0, 0, 0, 0


    accuracy = accuracy / 16
    precision = precision / (16-precisionerror)
    recall = recall / (16-recallerror)

    precisionerrorlist.append(precisionerror)
    recallerrorlist.append(recallerror)
    return [accuracy, precision, recall]

We got all the necessary functions and sturctures to test our model. All we need to is run our code with 5-cv. For that reason we need to implement a cv function and a start function.

split_data_16p(A.K.A 5-cv): This function splits the data into 5 pieces. After that combines these pieces with each other and creates test and training datas. After creating these datas the function returns these in an array.


In [18]:
# split data into training and test sets 5 cross validation
def split_data_16p(df):
    # split data into 5 pieces for 10000 samples

    df1_split = df[0:2000]
    df2_split = df[2000:4000]
    df3_split = df[4000:6000]
    df4_split = df[6000:8000]
    df5_split = df[8000:10000]

    # split into training and test sets
    df1_train = np.concatenate((df2_split, df3_split, df4_split, df5_split), axis=0)
    df1_test = df1_split

    df2_train = np.concatenate((df1_split, df3_split, df4_split, df5_split), axis=0)
    df2_test = df2_split

    df3_train = np.concatenate((df1_split, df2_split, df4_split, df5_split), axis=0)
    df3_test = df3_split

    df4_train = np.concatenate((df1_split, df2_split, df3_split, df5_split), axis=0)
    df4_test = df4_split

    df5_train = np.concatenate((df1_split, df2_split, df3_split, df4_split), axis=0)
    df5_test = df5_split

    #return a 2d list that contains the training and test sets
    return [[df1_train, df1_test], [df2_train, df2_test], [df3_train, df3_test], [df4_train, df4_test], [df5_train, df5_test]]



Now we need to run our code.

start16P: 1-) We shuffle our data
          2-) We split our data to 5 different train and test data pairs with the split_data_ee function.
          3-) After that we get our first test-training pair in a for loop.
          4-) We choosed our pair . Now we need to choose our closest neighbor paremeter k for KNN. We implemented               a for loop that starts from 1 and counts all the odd numbers which is smaller than 10.(k = 1,3,5,7,9)
          5-) We got the k value now we can call KNN function. We called the KNN function and got the predicted                   values and weightedpredicted values(We explained earlier that our knn function calculates same at                   the same time).
          6-) After that we need actual values to calculate accuracy,precision,recall. We retrieve the actual                     values with the help of a function that we created before :get_class_type.
          7-) We got the actual values, we got the predicted values now we can calculate accuracy,precision,recall.
          8-) We calculate accuracy,precision,recall for non-weighted predictions. And we append them to the                     all_values list.
          9-) We calculate accuracy,precision,recall for weighted predictions. And we append them to the all_values               list.
          10-) Above states repeated until the each train-test pair and each k value combined.
          11-) After getting all the accuracy,precision,recall for non-normaliazed data we repeat the same states                  for normalized data.
          12-) Normalize the data.
          13-) Repeat the first ten steps for normalized data.

In [19]:
def start16P(df):
    df = shuffle(df, len(df))

    data_sets = split_data_16p(df)
    # normalize the data

    for i in range(5):
        train_set = data_sets[i][0]
        test_set = data_sets[i][1]

        for j in range(1, 10, 2):
            #k-nn classifier
            personalityKNN(train_set, test_set, j)
            actual_classes = convert_classes_to_numbers(get_class_type(test_set))
            predicted_classes = convert_classes_to_numbers(predictionlist)
            predicted_classes_weighted = convert_classes_to_numbers(weightedpredictionlist)

            #knn confusion matrix
            confusion_matrix = place_in_confusion_matrix(actual_classes, predicted_classes)
            accuracy_precision_recall = performance_matrix(confusion_matrix)
            all_values[i].append(accuracy_precision_recall)

            #weighted knn confusion matrix
            confusion_matrix1 = place_in_confusion_matrix(actual_classes, predicted_classes_weighted)
            accuracy_precision_recall_weighted = performance_matrix(confusion_matrix1)
            all_values[i].append(accuracy_precision_recall_weighted)

    normalized_data = min_max_normalization_16p(df)
    normalized_data_sets = split_data_16p(normalized_data)
    for i in range(5):
        normalized_train_data = normalized_data_sets[i][0]
        normalized_test_data = normalized_data_sets[i][1]
        for j in range(1, 10, 2):
            #normalized knn classifier
            personalityKNN(normalized_train_data, normalized_test_data, j)
            actual_classes_normalized = convert_classes_to_numbers(get_class_type(test_set))
            predicted_classes_normalized = convert_classes_to_numbers(predictionlist)
            predicted_classes_weighted_normalized = convert_classes_to_numbers(weightedpredictionlist)

            #normalized knn confusion matrix
            confusion_matrix2 = place_in_confusion_matrix(actual_classes_normalized, predicted_classes_normalized)
            accuracy_precision_recall_normalized = performance_matrix(confusion_matrix2)
            all_values[i].append(accuracy_precision_recall_normalized)

            #normalized weighted knn confusion matrix
            confusion_matrix3 = place_in_confusion_matrix(actual_classes_normalized, predicted_classes_weighted_normalized)
            accuracy_precision_recall_weighted_normalized = performance_matrix(confusion_matrix3)
            all_values[i].append(accuracy_precision_recall_weighted_normalized)

In [20]:
start16P(df)

Rows printed 0 to 4 corresponds to 5 folds. 

Even numbers between 0 to 9 represents non-weighted , non-normalized KNN results that are ordered by KNN closest neighbor k values.They are ordered in a increasing order.

Odd numbers between 0 to 9 represents weighted , non-normalized KNN results that are ordered by KNN closest neighbor k values.They are ordered in a increasing order.

Even numbers between 10 to 19 represents non-weighted , normalized KNN results that are ordered by KNN closest neighbor k values.They are ordered in a increasing order.

Odd numbers between 10 to 19 represents weighted , normalized KNN results that are ordered by KNN closest neighbor k values.They are ordered in a increasing order

In [21]:
for i in range(len(all_values)):
    for j in range(len(all_values[i])):
        for l in range(len(all_values[i][j])):
            all_values[i][j][l] = round(all_values[i][j][l], 4)
pd.DataFrame(all_values)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,"[0.9989, 0.9916, 0.9918]","[0.9989, 0.9916, 0.9918]","[0.9981, 0.9849, 0.9856]","[0.9991, 0.9923, 0.9927]","[0.9979, 0.9837, 0.9841]","[0.9985, 0.9881, 0.9882]","[0.9979, 0.9838, 0.9841]","[0.9984, 0.9871, 0.9874]","[0.9981, 0.9849, 0.985]","[0.9984, 0.9876, 0.988]","[0.8821, 0.0564, 0.0565]","[0.8821, 0.0564, 0.0565]","[0.8822, 0.0577, 0.0581]","[0.8821, 0.0563, 0.0566]","[0.8821, 0.0561, 0.0566]","[0.882, 0.0557, 0.0561]","[0.8818, 0.0544, 0.0545]","[0.8819, 0.0549, 0.055]","[0.8818, 0.0544, 0.0545]","[0.8818, 0.0544, 0.0545]"
1,"[0.9991, 0.9927, 0.9924]","[0.9991, 0.9927, 0.9924]","[0.9983, 0.9868, 0.9866]","[0.9989, 0.9919, 0.9915]","[0.9982, 0.9863, 0.986]","[0.9987, 0.9902, 0.99]","[0.9977, 0.9818, 0.9817]","[0.9988, 0.9907, 0.9904]","[0.9976, 0.9807, 0.9809]","[0.9987, 0.9894, 0.9897]","[0.8829, 0.0631, 0.0638]","[0.8829, 0.0631, 0.0638]","[0.883, 0.0637, 0.0642]","[0.883, 0.0637, 0.0642]","[0.8829, 0.0627, 0.0632]","[0.8829, 0.0627, 0.0632]","[0.8828, 0.0616, 0.0622]","[0.8827, 0.0616, 0.0622]","[0.8827, 0.061, 0.0617]","[0.8827, 0.061, 0.0617]"
2,"[0.9994, 0.9954, 0.9951]","[0.9994, 0.9954, 0.9951]","[0.9988, 0.9907, 0.9907]","[0.9994, 0.9957, 0.9954]","[0.9983, 0.9865, 0.9862]","[0.999, 0.9921, 0.992]","[0.9978, 0.9827, 0.9828]","[0.9986, 0.9894, 0.9891]","[0.998, 0.9842, 0.9843]","[0.9986, 0.9889, 0.9886]","[0.8828, 0.0626, 0.0621]","[0.8828, 0.0626, 0.0621]","[0.8828, 0.0625, 0.0621]","[0.8828, 0.0625, 0.0621]","[0.8828, 0.0625, 0.0622]","[0.8828, 0.062, 0.0617]","[0.8829, 0.0634, 0.0631]","[0.8829, 0.0629, 0.0626]","[0.8829, 0.0634, 0.0631]","[0.8829, 0.0634, 0.0631]"
3,"[0.9996, 0.9964, 0.9965]","[0.9996, 0.9964, 0.9965]","[0.9988, 0.9907, 0.9908]","[0.9996, 0.9965, 0.9965]","[0.9979, 0.9831, 0.9828]","[0.9991, 0.9931, 0.9933]","[0.9978, 0.9825, 0.9827]","[0.9986, 0.9889, 0.9888]","[0.9977, 0.9814, 0.9817]","[0.9987, 0.9895, 0.9895]","[0.8837, 0.0694, 0.0688]","[0.8837, 0.0694, 0.0688]","[0.8838, 0.0697, 0.0693]","[0.8838, 0.0698, 0.0693]","[0.8838, 0.07, 0.0697]","[0.8838, 0.07, 0.0697]","[0.8838, 0.0699, 0.0697]","[0.8838, 0.0699, 0.0697]","[0.8837, 0.0695, 0.0693]","[0.8838, 0.0695, 0.0693]"
4,"[0.9999, 0.9991, 0.9989]","[0.9999, 0.9991, 0.9989]","[0.9996, 0.9969, 0.9971]","[0.9999, 0.9995, 0.9995]","[0.9994, 0.9955, 0.9957]","[0.9997, 0.9974, 0.9975]","[0.9988, 0.9906, 0.9908]","[0.9996, 0.9965, 0.9966]","[0.9984, 0.9875, 0.988]","[0.9993, 0.9945, 0.9946]","[0.9996, 0.9964, 0.9964]","[0.9996, 0.9964, 0.9964]","[0.9987, 0.9892, 0.9894]","[0.9988, 0.9903, 0.9907]","[0.9988, 0.9898, 0.9905]","[0.9988, 0.9904, 0.991]","[0.9983, 0.9862, 0.9863]","[0.9986, 0.9891, 0.9892]","[0.9979, 0.9831, 0.9834]","[0.9988, 0.9899, 0.9903]"


In [22]:
print(precisionerrorlist)
print(recallerrorlist)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Printing all average values for 5 folds. As we can see from the printed table weighted-nonweighted, KNN closest neighbor k value , folds have actually really small effect on the data.This means that data is shuffled well and it is unbiased. 

On the other hand normalization changes the view. After normalization accuracy, precision and recall drops down significantly.Normalization is a way of taking data that is slightly dissimilar but giving it a common state.Sometimes normalizing data removes important feature differences therefore causing accuracy to go down. Other times, it helps to eliminate noise in your features which cause incorrect classifications. The problem might be KNN overfit, which is to say it memorized the data very well, but does not work well at all on new data. We can try to understand wheter it is overfiting or not using validation. When we look at the printed table again we can see that with folds accuracy , precision and recall values increasing steadly. From these results we can assume that normalization caused overfiting problem.



In [23]:
average_values = []
average_accuracy = 0
average_precision = 0
average_recall = 0
    
for j in range(len(all_values[i])):
    for i in range(len(all_values)):
        average_accuracy += all_values[i][j][0]
        average_precision += all_values[i][j][1]
        average_recall += all_values[i][j][2]
    average_accuracy = average_accuracy/5
    average_precision = average_precision/5
    average_recall = average_recall/5
    average_values.append([average_accuracy, average_precision, average_recall])
    average_accuracy = 0
    average_precision = 0
    average_recall = 0

pd.DataFrame(average_values)
    

,0,1,2
0,0.99938,0.99504,0.99494
1,0.99938,0.99504,0.99494
2,0.99872,0.99000,0.99016
3,0.99938,0.99518,0.99512
4,0.99834,0.98702,0.98696
5,0.99900,0.99218,0.99220
6,0.99800,0.98428,0.98442
7,0.99880,0.99052,0.99046
8,0.99796,0.98374,0.98398
9,0.99874,0.98998,0.99008


PART 2:

    PROBLEM DEFINITION
    
Here we have 2 different values that we need to estimate . We will predict these values by looking to the sample's 8 different attributes. After calculating these 2 different values we will test our model with taking MAEs of these result. 

We are reading the 2nd data set and converting them to values

In [123]:
#part 2

dfEE = pd.read_csv('energy_efficiency_data.csv', encoding='cp1252')
dfEE = dfEE.values

After reading the data set we are declaring the global variables that will be used in the second part.

all_values_ee : This is the data sturcture that holds every Mean Absolute Error for every combinations that will be done in this project.

predictionlistee: This is the data sturcture that will be filled with arrays of predicted heating_load and cooling_load values while the KNN algortihm is working.

weightedpredictionlistee: This is the data sturcture that will be filled with arrays of predicted weighted heating and cooling values while the KNN algortihm is working.

For both normal and weighted KNN there is only one function that calculates the normal and weighted predictions. After calculations it puts the values to corresponding data structure

In [124]:
global all_values_ee
all_values_ee = [[] for l in range(5)]

global predictionlistee , weightedpredictionlistee
predictionlistee = [[None for m in range(2)] for p in range(156)]
weightedpredictionlistee = [[None for n in range(2)] for t in range(156)]

For this project we had to shuflle the data to reach unbiased results. For that reason we had to implement a shuffle function that is not a built-in function as said in project description. We used a shuffle techniueqe called fisher-yates shuffle.

Funny note : for some reasons there were some duplicated values in our data set. We were curious about this error and we implemented a duplucation counter method. After some try and error we have fixed the issue :))

In [125]:
#shuffle function from scratch
def shuffle(aux, n):
    #shuffle
    for i in range(n):
        #generate random index
        index = np.random.randint(0, len(aux))
        #swap
        aux[i], aux[index] = aux[index], aux[i]

    count_duplicates(aux)
    return aux

#count duplicates
def count_duplicates(set):
    #count duplicates
    count = 0
    for i in range(len(set)-1):
        if set[i][0] == set[i+1][0]:
            count += 1
    return count

There are the functions that we used to make a complete KNN function for regression.

eePredict : This function takes a training_Data , a distance list and a k-value for closest neighbors.After taking these values it calculates heatingLoad and coolingLoad and returns them in an array.

eePredictWeighted: This is function does the same thing with the eePredict. The only difference between them is eePredictWeighted calculates with weighted distances.

eeKNN: This function calculates the distance list that will be used in eePredict and eePredictWeighted. After calculating the distance list , eeKNN function will call eePredict and eePredictWeighted and store their results in
predictionlistee and weightedpredictionlistee.

In [126]:
def eePredict(training_Data, distancelist, k):
    heatingload = 0.0
    coolingload = 0.0
    for i in range(0, k):
        heatingload += training_Data[distancelist[i][1], 8]
        coolingload += training_Data[distancelist[i][1], 9]

    heatingload = heatingload / k
    coolingload = coolingload / k

    return [heatingload, coolingload]


def eePredictWeighted(training_Data, distancelist, k):
    heatingload = 0
    coolingload = 0
    weights = 0.0

    for i in range(0, k):
        heatingload += (1/distancelist[i][0]) * training_Data[distancelist[i][1], 8]
        coolingload += (1/distancelist[i][0]) * training_Data[distancelist[i][1], 9]
        weights += 1/distancelist[i][0]

    heatingload = heatingload / weights
    coolingload = coolingload / weights

    return [heatingload, coolingload]

def eeKNN (training_Data, test_Data, k):
    distance = 0.0
    global predictionlistee, weightedpredictionlistee

    #create predictionlistee and weightedpredictionlistee 2d lists to store the predicted classes 2columns and len(test_Data) rows
    distancelist = [None] * len(training_Data)
    predictionlistee = [[None for m in range(2)] for p in range(len(test_Data))]
    weightedpredictionlistee = [[None for n in range(2)] for t in range(len(test_Data))]
    for i in range(len(test_Data)):
        for j in range(len(training_Data)):
            for l in range(0, 8):
                distance += (test_Data[i, l] - training_Data[j, l])**2
            distance = math.sqrt(distance)
            distancelist[j] = [distance, j]
        distancelist.sort()
        predictionlistee[i] = eePredict(training_Data, distancelist, k)
        weightedpredictionlistee[i] = eePredictWeighted(training_Data, distancelist, k)

For this project we test our model with raw and normalized data. For raw part we didn't have to do anything. But for normalization we implemented our normalization method. As said in the project description we implemented a min-max normalization method.

In [127]:
def min_max_normalization_ee(df):
    min, max = 0, 0
    for i in range(0, 8):
        for j in df:
            if j[i] > max:
                max = j[i]
            elif j[i] < min:
                min = j[i]
        for l in df:
            l[i] = (l[i] - min) / (max - min)
        min, max = 0, 0
    return df

To compare our results with the actual values we had to get the actual values. For that we created 2 functions.


get_heating_value_ee: This function takes a numpy array. After that it retrieves the heating values of that given set and return them in an array.

get_cooling_value_ee: This function does the same thing with the get_heating_value_ee but it return cooling values instead of heating values.


After getting the actual values we need method that compares actual and predicted values. After comparing it needs to calculate mean absolute error.

calculate_mean_absolute_error: This function takes 2 parameters called actual_values, predicted_values. After that it calculates mean absolute error for given sets and returns it.

In [128]:
def get_heating_value_ee(set):
    values = []
    for i in range(len(set)):
        values.append(set[i][8])
    return values

def get_cooling_value_ee(set):
    values = []
    for i in range(len(set)):
        values.append(set[i][9])
    return values



def calculate_mean_absolute_error(actual_values, predicted_values):
    sum = 0.0
    for i in range(0, len(actual_values)):
        sum += abs(actual_values[i] - predicted_values[i])
    return sum / len(actual_values)

We got all the necessary functions and sturctures to test our model. All we need to is run our code with 5-cv. For that reason we need to implement a cv function and a start function.

split_data_ee(A.K.A 5-cv): This function splits the data into 5 pieces. After that combines these pieces with each other and creates test and training datas. After creating these datas the function returns these in an array.


Now we need to run our code.

start_ee: 1-) We shuffle our data
          2-) We split our data to 5 different train and test data pairs with the split_data_ee function.
          3-) After that we get our first test-training pair in a for loop.
          4-) We choosed our pair . Now we need to choose our closest neighbor paremeter k for KNN. We implemented               a for loop that starts from 1 and counts all the odd numbers which is smaller than 10.(k = 1,3,5,7,9)
          5-) We got the k value now we can call KNN function. We called the KNN function and got the predicted                   values and weightedpredicted values(We explained earlier that our knn function calculates same at                   the same time).
          6-) After that we need actual values to calculate MAEs. We retrieve the actual values with the help of                 functions that we wrote earlier(get_heating_value_ee and get_cooling_value_ee).
          7-) We got the actual values, we got the predicted values now we can calculate MAEs.
          8-) We calculate MAE for non-weighted predictions. And we append them to the all_values list.
          9-) We calculate MAE for weighted predictions. And we append them to the all_values list.
          10-) Above states repeated until the each train-test pair and each k value combined.
          11-) After getting all the MAEs for non-normaliazed data we repeat the same states for normalized data.
          12-) Normalize the data.
          13-) Repeat the first ten steps for normalized data.

In [129]:
def split_data_ee(set):
    # split data into 5 sets by 153 , 153 , 153 , 153 , 156
    df1 = set[0:153]
    df2 = set[153:306]
    df3 = set[306:459]
    df4 = set[459:612]
    df5 = set[612:768]

    # split them into training and test sets
    training_set1 = np.concatenate((df1, df2, df3, df4), axis=0)
    test_set1 = df5
    training_set2 = np.concatenate((df1, df2, df3, df5), axis=0)
    test_set2 = df4
    training_set3 = np.concatenate((df1, df2, df4, df5), axis=0)
    test_set3 = df3
    training_set4 = np.concatenate((df1, df3, df4, df5), axis=0)
    test_set4 = df2
    training_set5 = np.concatenate((df2, df3, df4, df5), axis=0)
    test_set5 = df1

    # return a 2d list that contains the training and test sets
    return [[training_set1, test_set1], [training_set2, test_set2], [training_set3, test_set3], [training_set4, test_set4], [training_set5, test_set5]]

def start_ee(df):

    dfEE = shuffle(df, len(df))

    data_sets_ee = split_data_ee(dfEE)

    for i in range(5):
        training_set = data_sets_ee[i][0]
        test_set = data_sets_ee[i][1]

        for j in range(1,10,2):
            eeKNN(training_set, test_set, j)

            actual_heating_values = get_heating_value_ee(test_set)
            actual_cooling_values = get_cooling_value_ee(test_set)

            predicted_heating_values = []
            predicted_cooling_values = []

            for k in range(len(predictionlistee)):
                predicted_heating_values.append(predictionlistee[k][0])
                predicted_cooling_values.append(predictionlistee[k][1])


            heating_mae = calculate_mean_absolute_error(actual_heating_values, predicted_heating_values)
            cooling_mae = calculate_mean_absolute_error(actual_cooling_values, predicted_cooling_values)


            mean_absolute_errors = [heating_mae, cooling_mae]

            all_values_ee[i].append(mean_absolute_errors)



            # do all the same for the weighted prediction listee
            predicted_heating_values_weighted = []
            predicted_cooling_values_weighted = []

            for l in range(len(weightedpredictionlistee)):
                predicted_heating_values_weighted.append(weightedpredictionlistee[l][0])
                predicted_cooling_values_weighted.append(weightedpredictionlistee[l][1])

            heating_mae_weighted = calculate_mean_absolute_error(actual_heating_values, predicted_heating_values_weighted)
            cooling_mae_weighted = calculate_mean_absolute_error(actual_cooling_values, predicted_cooling_values_weighted)

            mean_absolute_errors_weighted = [heating_mae_weighted, cooling_mae_weighted]

            all_values_ee[i].append(mean_absolute_errors_weighted)

            #do all the same for the normalized data


    normalized_data_ee = min_max_normalization_ee(dfEE)
    normalized_data_sets_ee = split_data_ee(normalized_data_ee)
    for i in range(5):
        normalized_training_set = normalized_data_sets_ee[i][0]
        normalized_test_set = normalized_data_sets_ee[i][1]
        for j in range(1,10,2):

            eeKNN(normalized_training_set, normalized_test_set, j)

            predicted_heating_values_normalized = []
            predicted_cooling_values_normalized = []

            for m in range(len(predictionlistee)):
                predicted_heating_values_normalized.append(predictionlistee[m][0])
                predicted_cooling_values_normalized.append(predictionlistee[m][1])

            heating_mae_normalized = calculate_mean_absolute_error(actual_heating_values, predicted_heating_values_normalized)
            cooling_mae_normalized = calculate_mean_absolute_error(actual_cooling_values, predicted_cooling_values_normalized)

            mean_absolute_errors_normalized = [heating_mae_normalized, cooling_mae_normalized]

            all_values_ee[i].append(mean_absolute_errors_normalized)

            #do all the same for the normalized weighted data

            predicted_heating_values_normalized_weighted = []
            predicted_cooling_values_normalized_weighted = []

            for n in range(len(weightedpredictionlistee)):
                predicted_heating_values_normalized_weighted.append(weightedpredictionlistee[n][0])
                predicted_cooling_values_normalized_weighted.append(weightedpredictionlistee[n][1])

            heating_mae_normalized_weighted = calculate_mean_absolute_error(actual_heating_values, predicted_heating_values_normalized_weighted)
            cooling_mae_normalized_weighted = calculate_mean_absolute_error(actual_cooling_values, predicted_cooling_values_normalized_weighted)

            mean_absolute_errors_normalized_weighted = [heating_mae_normalized_weighted, cooling_mae_normalized_weighted]

            all_values_ee[i].append(mean_absolute_errors_normalized_weighted)

We are running the code.

In [130]:
start_ee(dfEE)

We print our all_values_ee that contains every combination of k folds, weighted data, KNN closest neighbor k, normalized and non-normalized data.



Rows printed 0 to 4 corresponds to 5 folds. 

Even numbers between 0 to 9 represents non-weighted , non-normalized KNN results that are ordered by KNN closest neighbor k values.They are ordered in a increasing order.

Odd numbers between 0 to 9 represents weighted , non-normalized KNN results that are ordered by KNN closest neighbor k values.They are ordered in a increasing order.

Even numbers between 10 to 19 represents non-weighted , normalized KNN results that are ordered by KNN closest neighbor k values.They are ordered in a increasing order.

Odd numbers between 10 to 19 represents weighted , normalized KNN results that are ordered by KNN closest neighbor k values.They are ordered in a increasing order.


In [131]:
for i in range(len(all_values_ee)):
    for j in range(len(all_values_ee[i])):
        for l in range(len(all_values_ee[i][j])):
            all_values_ee[i][j][l] = round(all_values_ee[i][j][l], 4)
pd.DataFrame(all_values_ee)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,"[1.9621, 2.0179]","[1.9621, 2.0179]","[1.7027, 1.7665]","[1.7095, 1.7554]","[1.6004, 1.786]","[1.6103, 1.7726]","[1.5374, 1.7231]","[1.5649, 1.7321]","[1.5125, 1.7215]","[1.5435, 1.7308]","[12.0941, 11.2536]","[12.0941, 11.2536]","[11.8657, 10.875]","[11.8665, 10.8738]","[11.9726, 10.9753]","[11.9711, 10.9725]","[12.0108, 11.0425]","[12.009, 11.0386]","[11.9398, 11.0278]","[11.9397, 11.0257]"
1,"[1.7675, 2.1894]","[1.7675, 2.1894]","[1.5821, 1.8471]","[1.5775, 1.8412]","[1.5002, 1.8142]","[1.5028, 1.8088]","[1.4843, 1.8262]","[1.4818, 1.8104]","[1.4398, 1.7679]","[1.4605, 1.7767]","[11.2963, 10.4776]","[11.2963, 10.4776]","[11.0197, 10.2972]","[11.0203, 10.2947]","[11.0845, 10.306]","[11.0841, 10.3043]","[11.12, 10.3269]","[11.1178, 10.3246]","[11.1215, 10.3714]","[11.1202, 10.3686]"
2,"[1.595, 1.978]","[1.595, 1.978]","[1.6979, 1.9057]","[1.6576, 1.8936]","[1.5561, 1.8108]","[1.5507, 1.8233]","[1.6274, 1.8193]","[1.6115, 1.827]","[1.625, 1.7805]","[1.6045, 1.7931]","[10.6687, 10.14]","[10.6687, 10.14]","[10.6411, 10.0031]","[10.6415, 10.0032]","[10.674, 9.9965]","[10.6738, 9.9974]","[10.6865, 9.9343]","[10.6862, 9.9355]","[10.7372, 10.0255]","[10.7365, 10.0244]"
3,"[1.8137, 2.0976]","[1.8137, 2.0976]","[1.8624, 1.9353]","[1.8314, 1.923]","[1.8278, 1.904]","[1.81, 1.9021]","[1.8156, 1.9131]","[1.7954, 1.9044]","[1.7755, 1.9971]","[1.772, 1.9735]","[9.8218, 9.0127]","[9.8218, 9.0127]","[9.9244, 8.9434]","[9.9087, 8.9267]","[9.9779, 9.0233]","[9.9642, 9.0087]","[9.9761, 9.0106]","[9.9666, 9.0009]","[9.9514, 8.9978]","[9.9442, 8.9901]"
4,"[2.2546, 1.9733]","[2.2546, 1.9733]","[1.858, 1.6645]","[1.8724, 1.6494]","[1.8133, 1.6861]","[1.8107, 1.646]","[1.8362, 1.6802]","[1.822, 1.6406]","[1.8013, 1.735]","[1.8011, 1.6798]","[2.5498, 2.5253]","[2.5498, 2.5253]","[2.3429, 2.5426]","[2.3507, 2.5492]","[2.2353, 2.4935]","[2.2417, 2.4978]","[2.2415, 2.3989]","[2.2469, 2.4047]","[2.2085, 2.4101]","[2.2136, 2.4145]"


After printing all the results we print the average values of folds as a table. And we print the every combination for heating and cooling value.

Printing all average values for 5 folds. As we can see from the printed table weighted-nonweighted, KNN closest neighbor k value , folds have actually really small effect on the data.This means that data is shuffled well and it is unbiased. 

On the other hand normalization changes the view. After normalization average values for heating and cooling MAE drops down significantly.Normalization is a way of taking data that is slightly dissimilar but giving it a common state.Sometimes normalizing data removes important feature differences therefore causing accuracy to go down. Other times, it helps to eliminate noise in your features which cause incorrect classifications. The problem might be KNN overfit, which is to say it memorized the data very well, but does not work well at all on new data. We can try to understand wheter it is overfiting or not using validation. When we look at the printed table again we can see that with folds heating and cooling MAE values increasing steadly. From these results we can assume that normalization caused overfiting problem.

In [132]:
average_values_ee = []
average_heating = 0
average_cooling = 0
    
for j in range(len(all_values_ee[i])):
    for i in range(len(all_values_ee)):
        average_heating += all_values_ee[i][j][0]
        average_cooling += all_values_ee[i][j][1]
    average_heating = average_heating/5
    average_cooling = average_cooling/5
    average_values_ee.append([average_heating, average_cooling])
    average_heating = 0
    average_cooling = 0

pd.DataFrame(average_values_ee)

,0,1
0,1.87858,2.05124
1,1.87858,2.05124
2,1.74062,1.82382
3,1.72968,1.81252
4,1.65956,1.80022
5,1.65690,1.79056
6,1.66018,1.79238
7,1.65512,1.78290
8,1.63082,1.80040
9,1.63632,1.79078


            RESOURCES
https://computersciencewiki.org/index.php/Normalization,
https://stackoverflow.com/questions/42092448/accuracy-difference-on-normalization-in-knn
